In [1]:
import asyncio


In [1]:
xmlTemplate = '<?xml version="1.0" encoding="UTF-8"?><NLRIn source="NARSIM" xmlns:sti="http://www.w3.org/2001/XMLSchema-instance"><truth><callsign>{callsign}</callsign><ssr_s>877777</ssr_s><lat unit="deg">{lat}</lat>'\
    '<lon unit="deg">{lon}</lon><alt unit="ft">{alt}</alt><gspd unit="ms">{gspd}</gspd><crs unit="degrees">{crs}</crs><v_rate unit="ms">{v_rate}</v_rate><turn_rate>{turn_rate}</turn_rate></truth></NLRIn>'

In [17]:
import socket
import time
import numpy as np
from SimConnect import *
from SimConnect.Enum import *
import configparser

# Constants
KNOTS_TO_METER_PER_SEC =  0.51444

# config for e.g. conenctions
config = configparser.ConfigParser()
config.read('./configuration.conf')

xmlTemplate = '<?xml version="1.0" encoding="UTF-8"?><NLRIn source="NARSIM" xmlns:sti="http://www.w3.org/2001/XMLSchema-instance"><truth><callsign>{callsign}</callsign><ssr_s>877777</ssr_s><lat unit="deg">{lat}</lat>'\
    '<lon unit="deg">{lon}</lon><alt unit="ft">{alt}</alt><gspd unit="ms">{gspd}</gspd><crs unit="degrees">{crs}</crs><v_rate unit="ms">{v_rate}</v_rate><turn_rate>{turn_rate}</turn_rate></truth></NLRIn>'
# Lägg till v_rate och turn_rate från sim


# TODO
xmlFlightplan = '''<?xml version="1.0" encoding="UTF-8"?>
<NLRIn source="NARSIM" xmlns:sti="http://www.w3.org/2001/XMLSchema-instance"><flightplan><callsign>SAS123</callsign><ssr_s>877777</ssr_s><clr_fl unit="ft">5000</clr_fl></flightplan></NLRIn>'''

'''
MSFS event variable names for SimConnect
Confirmed and working as of 29 nov 2022. Unit as received from SimConnect.
'''
lat_varname = 'PLANE_LATITUDE'# [degrees]
lon_varname = 'PLANE_LONGITUDE'# [degrees]
alt_varname = 'PLANE_ALTITUDE'# [ft]
gspd_varname = 'GROUND_VELOCITY'# [kts]
crs_varname = 'PLANE_HEADING_DEGREES_TRUE'# [radians]
v_rate_varname = 'VELOCITY_BODY_Y' # [feet per second]
turn_rate_varname = 'ROTATION_VELOCITY_BODY_Y' # [feet per second] ??? Simconnect API docs wrong??
long_acc_varname = 'ACCELERATION_BODY_Z' #  [feet per second2]
v_acc_varname = 'ACCELERATION_BODY_Y' # [feet per second2]


lat_finder = lon_finder = alt_finder = gspd_finder = crs_finder = v_rate_finder = turn_rate_finder = ''
toa = lat = lon = alt = gspd = crs = v_rate = turn_rate = long_acc = v_acc = 0

# connections
NARSIM_IP = config['NARSIM']['IP']
NARSIM_PORT = config['NARSIM']['PORT']

# Time interval of solo flight feeder
time_interval_feeder = 1/float(config['solo-flight-feeder']['frequency'])


# Initialize all as False
MSFS_is_connected = False
MSFS_data_stream_is_initialized = False
NARSIM_is_connected = False
# Set to False for debug with MSFS only
send_data_to_NARSIM = False

# flight plan
callsign = config['flightplan']['callsign']

# think of smarter way to do this
def connect_MSFS() -> AircraftRequests:
    try:
        sm = SimConnect() # Object for Connection to MSFS
        MSFS_is_connected = True
        print('MSFS successfully connected!')
        ar = AircraftRequests(sm) # Object for requesting MSFS variables
        return ar, MSFS_is_connected

    except (ConnectionError, OSError):
        print('Unable to establish connection with MSFS. Make sure MSFS is running.')
        input('Press enter to attempt new connection.')

def connect_NARSIM(s) -> bool:
    try:
        s.connect((NARSIM_IP, NARSIM_PORT))
        print('NARSIM successfully connected!')
        NARSIM_is_connected = True
        return NARSIM_is_connected
    except socket.error as msg:
        print(f'Unable to connect to NARSIM, TCP socket error: ' + msg)



In [18]:
# Initialize time.
last_time = time.time()

# Main loop
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    
    while True:
        if not MSFS_is_connected:
            ar, MSFS_is_connected = connect_MSFS()

        if (not NARSIM_is_connected) and send_data_to_NARSIM:
            NARSIM_is_connected = connect_NARSIM(s)
        
        current_time = time.time()
        if (current_time >= last_time + time_interval_feeder):
            last_time = current_time
            toa = last_time

            if not MSFS_data_stream_is_initialized:
                try:
                    lat_finder = ar.find(lat_varname)
                    lon_finder = ar.find(lon_varname)
                    alt_finder = ar.find(alt_varname)
                    gspd_finder = ar.find(gspd_varname)
                    crs_finder = ar.find(crs_varname)
                    v_rate_finder = ar.find(v_rate_varname)
                    turn_rate_finder = ar.find(turn_rate_varname)
                    long_acc_finder = ar.find(long_acc_varname)
                    v_acc_finder = ar.find(v_acc_varname)

                    MSFS_data_stream_is_initialized = True
                except (ConnectionError, OSError):
                    print('Lost connection with MSFS. Unable to send TCP packet.')

            if MSFS_data_stream_is_initialized:
                try:
                    lat = lat_finder.get()
                    lon = lon_finder.get()
                    alt = alt_finder.get()
                    gspd = gspd_finder.get() * KNOTS_TO_METER_PER_SEC # send m/s to Narsim
                    crs = crs_finder.get()
                    v_rate = v_rate_finder.get()
                    turn_rate = turn_rate_finder.get()
                    long_acc = long_acc_finder.get()
                    v_acc = v_acc_finder.get()

                    translation_dict = {'callsign':callsign, 'toa':toa, 'lat':lat, 'lon':lon, 'alt':alt, 'gspd':gspd, 'crs':crs, 'v_rate':v_rate, 'turn_rate':turn_rate, 'long_acc':long_acc, 'v_acc':v_acc}
                    xml_output = xmlTemplate.format(**translation_dict)
                    #print(xml_output)
                    print(translation_dict)

                    try:
                        if send_data_to_NARSIM:
                            s.send(xml_output.encode())
                    except socket.error as msg:
                        print(f'Lost connection to NARSIM, TCP socket error: ' + msg)
                except (ConnectionError, OSError):
                    print('Lost connection with MSFS.')
        time.sleep(0.05)

MSFS successfully connected!
{'callsign': 'SAS123', 'toa': 1669752923.5388896, 'lat': 55.84668874815252, 'lon': 12.910687866035222, 'alt': 3776.1548444931755, 'gspd': 116.10361393683779, 'crs': 5.839726880028089, 'v_rate': -21.977177851807756, 'turn_rate': 0.0004814625671218367, 'long_acc': -0.20894341534684507, 'v_acc': -0.340988411213941}
{'callsign': 'SAS123', 'toa': 1669752923.8785567, 'lat': 55.84699729799737, 'lon': 12.910399803107495, 'alt': 3775.7736991444945, 'gspd': 116.0850566113317, 'crs': 5.8397607564399365, 'v_rate': -22.120309154973512, 'turn_rate': 8.001956433373322e-06, 'long_acc': -0.2606532188547343, 'v_acc': -0.6585309969117202}
{'callsign': 'SAS123', 'toa': 1669752924.2475584, 'lat': 55.8473418469741, 'lon': 12.910072897020711, 'alt': 3775.2082938997423, 'gspd': 116.05724259972125, 'crs': 5.839736777653858, 'v_rate': -22.691458989032, 'turn_rate': -0.0006733378169719655, 'long_acc': -0.24263653271181954, 'v_acc': 0.27376739920391246}
{'callsign': 'SAS123', 'toa': 1

KeyboardInterrupt: 

In [11]:
main()

UnboundLocalError: local variable 'MSFS_is_connected' referenced before assignment

In [23]:
XML_TEMPLATE_TRUTH = (
    '<?xml version="1.0" encoding="UTF-8"?><NLRIn source="NARSIM" xmlns:sti="http://www.w3.org/2001/XMLSchema-instance">'
    '<truth><callsign>{callsign}</callsign><ssr_s>877777</ssr_s><lat unit="deg">{lat}</lat>'
    '<lon unit="deg">{lon}</lon><alt unit="ft">{alt}</alt><gspd unit="ms">{gspd}</gspd>'
    '<crs unit="degrees">{crs}</crs><v_rate unit="ms">{v_rate}</v_rate>'
    '<turn_rate>{turn_rate}</turn_rate></truth></NLRIn>'
)

print(XML_TEMPLATE_TRUTH)

<?xml version="1.0" encoding="UTF-8"?><NLRIn source="NARSIM" xmlns:sti="http://www.w3.org/2001/XMLSchema-instance"><truth><callsign>{callsign}</callsign><ssr_s>877777</ssr_s><lat unit="deg">{lat}</lat><lon unit="deg">{lon}</lon><alt unit="ft">{alt}</alt><gspd unit="ms">{gspd}</gspd><crs unit="degrees">{crs}</crs><v_rate unit="ms">{v_rate}</v_rate><turn_rate>{turn_rate}</turn_rate></truth></NLRIn>


In [24]:
d = {'a':3, 'b':2}

In [ ]:
d